In [ ]:
import json
import pandas as pd
import glob
import os

def json_to_excel_table_from_folder(folder_path, excel_path):
    """
    Reads all .json files in a folder, concatenates them into one DataFrame
    with the structure expected by the bacterial response analysis code,
    and saves to Excel.
    """
    all_rows = []

    # Find all json files in given folder
    json_files = glob.glob(os.path.join(folder_path, "*.json"))

    if not json_files:
        print("❌ No JSON files found in", folder_path)
        return

    print(f"📂 Found {len(json_files)} JSON files in {folder_path}")
    frameCounter=1
    for json_file in json_files:
        frameCounter = frameCounter+1
        with open(json_file, 'r') as f:
            try:
                data = json.load(f)
            except json.JSONDecodeError as e:
                print(f"⚠️ Skipping {json_file}: JSON decode error ({e})")
                continue

        # Safety check: if file has a single object instead of a list
        if isinstance(data, dict):
            data = [data]

        for cell in data:
            frame = frameCounter
            x = cell.get("x", None)
            y = cell.get("y", None)
            aimP = cell.get("cytoplasmConcentrations", {}).get("P", None)
            aimR = cell.get("cytoplasmConcentrations", {}).get("R", None)*100
            rfp = cell.get("cytoplasmConcentrations", {}).get("F", None)*10
            ExAimp = cell.get("extracellularConcentrations", {}).get("P", None)*1000

            cell_type = cell.get("cytoplasmConcentrations", {}).get("producer", None)

         
            all_rows.append([frame, x, y, aimP, aimR,rfp, ExAimp, cell_type])

    # Create combined DataFrame
    df = pd.DataFrame(all_rows, columns=[
        "Frame", "X", "Y", "aimp Intensity", "aimr Intensity", "rfp Intensity", "extracellular", "Cell Type"
    ])

    # Save to Excel
    df.to_excel(excel_path, index=False)
    print(f"✅ Converted {len(df)} rows from {len(json_files)} files into: {excel_path}")


if __name__ == "__main__":
    # Folder containing all JSON files
    folder_with_json = "./data"   # change if needed
    output_excel = "Xylose1_1IPTG_cropped_cell_info.xlsx"
    json_to_excel_table_from_folder(folder_with_json, output_excel)


📂 Found 35 JSON files in ./data
✅ Converted 43528 rows from 35 files into: Xylose1_1IPTG_cropped_cell_info.xlsx
